# Simple concurrency examples

## A pipeline

### Using threads

In [33]:
import threading
import queue 

intermediate_queue = queue.Queue()
consumer_queue = queue.Queue()

def producer():
    for i in range(10):
        intermediate_queue.put(i) # Queue is thread safe
        print("produced", i)
    print("producer: done")

def intermediate():
    while True:
        i = intermediate_queue.get()
        consumer_queue.put(i+1)
        if i == 9:
            break
    print("intermediate: done") 

def consumer():
    while True: 
        i = consumer_queue.get()
        print("consumed", i)
        if i == 10:
            break
    print("consumer: done") 

p = threading.Thread(target=producer)
i = threading.Thread(target=intermediate)
c = threading.Thread(target=consumer)

p.start()
i.start()
c.start()

produced 0
produced 1
producedconsumed 1
consumed 2
consumed 3
 2
produced 3
produced 4
produced 5
produced 6
produced 7
produced 8
produced 9
producer: done
intermediate: done
consumed 4
consumed 5
consumed 6
consumed 7
consumed 8
consumed 9
consumed 10
consumer: done


### Using processes

In [34]:
import multiprocessing

intermediate_queue = multiprocessing.Queue()
consumer_queue = multiprocessing.Queue()

def producer():
    for i in range(10):
        intermediate_queue.put(i) # Queue is thread safe
        print("produced", i)
    print("producer: done")

def intermediate():
    while True:
        i = intermediate_queue.get()
        consumer_queue.put(i+1)
        if i == 9:
            break
    print("intermediate: done") 

def consumer():
    while True: 
        i = consumer_queue.get()
        print("consumed", i)
        if i == 10:
            break
    print("consumer: done") 

p = multiprocessing.Process(target=producer)
i = multiprocessing.Process(target=intermediate)
c = multiprocessing.Process(target=consumer)

p.start()
i.start()
c.start()

produced 0
produced 1
consumed 1
produced 2
consumed 2
produced 3
produced 4
consumed 3
produced 5
produced 6
consumed 4
produced 7
consumed 5
produced 8
consumed 6
produced 9
consumed 7
producer: done
intermediate: done
consumed 8
consumed 9
consumed 10
consumer: done


### Using coroutines

In [35]:
def producer(next_task):
    for i in range(10):
        next_task.send(i) 
        print("produced", i)
    next_task.close()
    print("producer: done")

def intermediate(next_task): 
    try: 
        while True: 
            i = (yield)
            next_task.send(i+1)
    except GeneratorExit: 
        print("intermediate: done") 

def consumer():
    try: 
        while True: 
            i = (yield) 
            print("consumed", i) 
    except GeneratorExit: 
        print("consumer: done") 

c = consumer()
c.__next__() # Advance until the first yield
i = intermediate(c)
i.__next__() # Advance until the first yield
producer(i)

consumed 1
produced 0
consumed 2
produced 1
consumed 3
produced 2
consumed 4
produced 3
consumed 5
produced 4
consumed 6
produced 5
consumed 7
produced 6
consumed 8
produced 7
consumed 9
produced 8
consumed 10
produced 9
intermediate: done
producer: done
